## Load and Explore the Dataset

In [9]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('../data/df_file.csv') 

label_to_category = {
    0: 'Politics',
    1: 'Sport',
    2: 'Technology',
    3: 'Entertainment',
    4: 'Business'
}

# Add a new category called 'Other' for unmatched texts
label_to_category[5] = 'Other'

category_to_label = {v: k for k, v in label_to_category.items()}

# Add a new column for category names
df['category'] = df['Label'].map(label_to_category)

# Display basic info
print("Dataset Overview:")
print(df.head())

print("\nDataset Shape:", df.shape)
print("\nColumn Names and Data Types:")
print(df.dtypes)

# Display class distribution with category names
print("\nClass Distribution:")
print(df['category'].value_counts())

Dataset Overview:
                                                Text  Label  category
0  Budget to set scene for election\n \n Gordon B...      0  Politics
1  Army chiefs in regiments decision\n \n Militar...      0  Politics
2  Howard denies split over ID cards\n \n Michael...      0  Politics
3  Observers to monitor UK election\n \n Minister...      0  Politics
4  Kilroy names election seat target\n \n Ex-chat...      0  Politics

Dataset Shape: (2225, 3)

Column Names and Data Types:
Text        object
Label        int64
category    object
dtype: object

Class Distribution:
category
Sport            511
Business         510
Politics         417
Technology       401
Entertainment    386
Name: count, dtype: int64


### Text Preprocessing

In [10]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = ''.join([char for char in text if char.isalpha() or char == ' '])
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing
df['cleaned_text'] = df['Text'].apply(preprocess_text)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)  
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['Label']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Train-Test Split

In [11]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training Data Shape:", X_train.shape)
print("Testing Data Shape:", X_test.shape)

Training Data Shape: (1780, 5000)
Testing Data Shape: (445, 5000)


## Train a Classification Model

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Train a Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_to_category.values()))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nAccuracy:", accuracy_score(y_test, y_pred))

Classification Report:


ValueError: Number of classes, 5, does not match size of target_names, 6. Try specifying the labels parameter